In [ ]:
# Cell 1: Setup & Install - COMPLETE WORKING VERSION

# Install all packages WITH torchvision (it's required!)
!pip install -q --upgrade torch torchvision transformers datasets imageio imageio-ffmpeg decord tqdm bitsandbytes accelerate
!pip install -q "qwen-vl-utils[decord]==0.0.8"

import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# Verify torchvision is installed
try:
    import torchvision
    print("Torchvision:", torchvision.__version__)
    print("✅ All dependencies installed correctly")
except ImportError:
    print("❌ Torchvision missing - this shouldn't happen!")

torch.set_float32_matmul_precision("high")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 35.7 MB/s eta 0:00:00
Torch: 2.9.0+cu128
CUDA available: True
GPU: Tesla T4
Torchvision: 0.24.0+cu128
✅ All dependencies installed correctly


/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


cell 2

In [ ]:
# --- Colab: Create folders ---
import os, json

ROOT = "/content/camerabench"
VIDS = f"{ROOT}/videos_gif_mp4"
OUTS = f"{ROOT}/outputs"

for p in (ROOT, VIDS, OUTS):
    os.makedirs(p, exist_ok=True)

print("Root:", ROOT)
print("Videos:", VIDS)
print("Outputs:", OUTS)

Root: /content/camerabench
Videos: /content/camerabench/videos_gif_mp4
Outputs: /content/camerabench/outputs


In [ ]:
# --- PATCH: re-define GIF→MP4 utils with even-dimension padding ---
import imageio, imageio.v3 as iio
import numpy as np, os, warnings
from typing import Tuple, List
from PIL import Image
from decord import VideoReader, cpu

def to_rgb_frame(arr: np.ndarray) -> np.ndarray:
    # PIL-safe conversion path
    if isinstance(arr, Image.Image):
        if arr.mode in ("RGBA", "LA"):
            arr = arr.convert("RGBA")
            bg = Image.new("RGBA", arr.size, (0, 0, 0, 0))
            bg.alpha_composite(arr)
            arr = bg.convert("RGB")
        else:
            arr = arr.convert("RGB")
        return np.array(arr, dtype=np.uint8)

    # NumPy path
    if arr.dtype != np.uint8:
        a = arr.astype(np.float32)
        a = np.clip(a, 0, 255)
        arr = a.astype(np.uint8)

    if arr.ndim == 2:  # gray -> RGB
        arr = np.stack([arr, arr, arr], axis=-1)

    if arr.shape[-1] == 4:  # RGBA -> RGB (alpha over black)
        rgb = arr[..., :3].astype(np.float32)
        alpha = (arr[..., 3:4].astype(np.float32) / 255.0)
        rgb = (rgb * alpha).astype(np.uint8)
        arr = rgb

    if arr.shape[-1] != 3:
        first = arr[..., 0]
        arr = np.stack([first, first, first], axis=-1).astype(np.uint8)

    return arr

def _ensure_same_size(frames: List[np.ndarray]) -> Tuple[List[np.ndarray], Tuple[int, int]]:
    """Resize all frames to the first frame's WxH (consistent encoder input)."""
    if not frames:
        return frames, (0, 0)
    h, w = frames[0].shape[:2]
    out = []
    for f in frames:
        if f.shape[0] != h or f.shape[1] != w:
            pil = Image.fromarray(f)
            pil = pil.resize((w, h), resample=Image.Resampling.BILINEAR)
            f = np.array(pil, dtype=np.uint8)
        out.append(f)
    return out, (h, w)

def _pad_to_even(frames: List[np.ndarray]) -> Tuple[List[np.ndarray], Tuple[int, int]]:
    """Pad frames on the right/bottom by 1 pixel if width or height is odd (needed for yuv420p)."""
    if not frames:
        return frames, (0, 0)
    h, w = frames[0].shape[:2]
    pad_h = h % 2
    pad_w = w % 2
    if pad_h == 0 and pad_w == 0:
        return frames, (h, w)

    H, W = h + pad_h, w + pad_w
    out = []
    for f in frames:
        canvas = np.zeros((H, W, 3), dtype=np.uint8)
        canvas[:h, :w, :] = f
        out.append(canvas)
    return out, (H, W)

def decord_ok(path: str) -> bool:
    try:
        if not (os.path.exists(path) and os.path.getsize(path) > 0):
            return False
        vr = VideoReader(path, ctx=cpu(0))
        if len(vr) < 1:
            return False
        _ = vr[0]
        return True
    except Exception:
        return False

def convert_gif_to_mp4(gif_path: str, mp4_path: str, fps: float = 8.0) -> bool:
    """
    Robust GIF→MP4:
      - normalize to RGB
      - force consistent WxH
      - **pad to even dims** for yuv420p
      - macro_block_size=1 to avoid implicit resizing
    """
    try:
        frames = []
        with imageio.get_reader(gif_path) as reader:
            for frame in reader:
                frames.append(to_rgb_frame(frame))
        if not frames:
            warnings.warn(f"No frames decoded from GIF: {gif_path}")
            return False

        frames, _ = _ensure_same_size(frames)
        frames, _ = _pad_to_even(frames)  # <-- critical fix for yuv420p

        imageio.mimsave(
            mp4_path,
            frames,
            fps=fps,
            macro_block_size=1,
            codec="libx264",
            format="FFMPEG",
            ffmpeg_params=["-pix_fmt", "yuv420p", "-movflags", "+faststart"],
        )
        return decord_ok(mp4_path)
    except Exception as e:
        warnings.warn(
            f"GIF→MP4 failed for {gif_path} -> {mp4_path}: {e}\n\n"
            "Tip: this is often due to odd frame sizes with yuv420p; padding to even dims usually fixes it."
        )
        return False

In [ ]:
# --- Colab: Load dataset & build manifest (download GIFs, convert to MP4) ---
import os, json, time, hashlib, requests
from datasets import load_dataset
from tqdm import tqdm

MANIFEST_PATH = f"{OUTS}/gif2mp4_manifest.json"

# Load split="test"
ds = load_dataset("syCen/CameraBench", split="test")

def safe_filename(url: str) -> str:
    """Make a deterministic, safe base name from URL."""
    h = hashlib.md5(url.encode("utf-8")).hexdigest()[:16]
    base = os.path.basename(url).split("?")[0]
    return f"{h}_{base}"

def download_gif(url: str, out_path: str, timeout=20, retries=3) -> bool:
    for i in range(retries):
        try:
            r = requests.get(url, stream=True, timeout=timeout)
            if r.status_code == 200:
                with open(out_path, "wb") as f:
                    for chunk in r.iter_content(chunk_size=1024 * 256):
                        if chunk:
                            f.write(chunk)
                return os.path.exists(out_path) and os.path.getsize(out_path) > 0
        except Exception:
            time.sleep(1.0 * (i + 1))
    return False

manifest = []
for idx, row in enumerate(tqdm(ds, desc="Building GIF→MP4 manifest")):
    gif_url = row.get("video", None) or row.get("Video", None)
    rel_path = row.get("path", None)  # relative mp4 path/id in dataset metadata
    if not gif_url:
        continue

    gif_name = safe_filename(gif_url)
    local_gif = os.path.join(VIDS, gif_name if gif_name.lower().endswith(".gif") else gif_name + ".gif")
    local_mp4 = os.path.splitext(local_gif)[0] + ".mp4"

    # Download if not present
    if not os.path.exists(local_gif):
        ok = download_gif(gif_url, local_gif)
        if not ok:
            # Skip adding broken downloads; entry records failure state
            manifest.append({
                "row_idx": idx, "rel_path": rel_path, "gif_url": gif_url,
                "local_gif": local_gif, "local_mp4": local_mp4, "status": "gif_download_failed"
            })
            continue

    # Convert to MP4 if needed
    if not os.path.exists(local_mp4):
        _ = convert_gif_to_mp4(local_gif, local_mp4, fps=8.0)

    status = "ok" if decord_ok(local_mp4) else "mp4_unreadable"
    manifest.append({
        "row_idx": idx,
        "rel_path": rel_path,
        "gif_url": gif_url,
        "local_gif": local_gif,
        "local_mp4": local_mp4,
        "status": status,
    })

# Save manifest
with open(MANIFEST_PATH, "w") as f:
    json.dump(manifest, f, indent=2)

print(f"Saved manifest: {MANIFEST_PATH}")
print("Total items:", len(manifest))
print("Readable MP4s:", sum(1 for m in manifest if m["status"] == "ok"))
print("\nSample manifest entry:")
print(json.dumps(next((m for m in manifest if m["status"] == "ok"), manifest[0] if manifest else {}), indent=2))

Building GIF→MP4 manifest:   8%|▊         | 82/1071 [00:01<00:33, 29.87it/s]/tmp/ipython-input-3367234834.py:31: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  alpha = (arr[..., 3:4].astype(np.float32) / 255.0)
/tmp/ipython-input-3367234834.py:32: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  rgb = (rgb * alpha).astype(np.uint8)
Building GIF→MP4 manifest: 100%|██████████| 1071/1071 [29:49<00:00,  1.67s/it]

Saved manifest: /content/camerabench/outputs/gif2mp4_manifest.json
Total items: 1071
Readable MP4s: 1063

Sample manifest entry:
{
  "row_idx": 0,
  "rel_path": "videos/-2uIa-XMJC0.5.3.mp4",
  "gif_url": "https://huggingface.co/datasets/syCen/CameraBench/resolve/main/videos_gif/-2uIa-XMJC0.5.3.gif",
  "local_gif": "/content/camerabench/videos_gif_mp4/3e1afda20f104270_-2uIa-XMJC0.5.3.gif",
  "local_mp4": "/content/camerabench/videos_gif_mp4/3e1afda20f104270_-2uIa-XMJC0.5.3.mp4",
  "status": "ok"
}


In [ ]:
# Cell 5: Load Model & Processor - ROBUST VERSION

import torch
from transformers import BitsAndBytesConfig, AutoProcessor

MODEL_ID = "chancharikm/qwen2.5-vl-7b-cam-motion"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
)

print(f"Loading {MODEL_ID}...")

# Load processor
print("Loading processor...")
processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
print("✅ Processor loaded")

# Try multiple model loading approaches
print("Loading model with 4-bit quantization...")

# Approach 1: Try Qwen2.5-VL specific class
try:
    from transformers.models.qwen2_5_vl.modeling_qwen2_5_vl import Qwen2_5_VLForConditionalGeneration
    print("Using Qwen2_5_VLForConditionalGeneration...")
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        attn_implementation="sdpa",
    )
    print("✅ Loaded with Qwen2.5-VL class")

except (ImportError, AttributeError) as e:
    print(f"Approach 1 failed: {e}")

    # Approach 2: Use AutoModelForVision2Seq
    try:
        from transformers import AutoModelForVision2Seq
        print("Using AutoModelForVision2Seq...")
        model = AutoModelForVision2Seq.from_pretrained(
            MODEL_ID,
            device_map="auto",
            quantization_config=bnb_config,
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            attn_implementation="sdpa",
        )
        print("✅ Loaded with AutoModelForVision2Seq")

    except Exception as e2:
        print(f"Approach 2 failed: {e2}")

        # Approach 3: Load with trust_remote_code (uses model's own code)
        print("Using trust_remote_code to load custom model code...")
        from transformers import AutoModelForCausalLM
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            device_map="auto",
            quantization_config=bnb_config,
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            attn_implementation="sdpa",
        )
        print("✅ Loaded with trust_remote_code")

# Verify model has generate method
if not hasattr(model, 'generate'):
    raise RuntimeError(f"Model type {type(model).__name__} does not have generate() method!")

model.eval()
for p in model.parameters():
    p.requires_grad_(False)

print(f"✅ Model loaded on {model.device}")
print(f"Model type: {type(model).__name__}")
print(f"Has generate: {hasattr(model, 'generate')}")
print("🎉 Ready to generate captions!")

Loading chancharikm/qwen2.5-vl-7b-cam-motion...
Loading processor...
✅ Processor loaded
Loading model with 4-bit quantization...
Using Qwen2_5_VLForConditionalGeneration...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

✅ Loaded with Qwen2.5-VL class
✅ Model loaded on cuda:0
Model type: Qwen2_5_VLForConditionalGeneration
Has generate: True
🎉 Ready to generate captions!


In [ ]:
# Cell 6: Sanity test - FIXED for Qwen2.5-VL

import time
import json
from qwen_vl_utils import process_vision_info

ROOT = "/content/camerabench"
OUTS = f"{ROOT}/outputs"
MANIFEST_PATH = f"{OUTS}/gif2mp4_manifest.json"

PROMPT_TXT = "Describe the camera motions."

# Load manifest
with open(MANIFEST_PATH, "r") as f:
    manifest = json.load(f)

test_items = [m for m in manifest if m["status"] == "ok"][:5]
print(f"Testing on {len(test_items)} videos\n")

def generate_caption_for_path(video_path: str):
    # Build messages
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "video", "video": video_path, "fps": 6.0, "max_frames": 32},
                {"type": "text", "text": PROMPT_TXT},
            ],
        }
    ]

    t0 = time.time()

    # Apply processor directly to messages (this is the key fix!)
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],  # Use the templated text
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )

    t_prep = time.time() - t0

    # Move to device
    inputs = inputs.to(model.device)

    t1 = time.time()
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=128,  # Increased for better captions
    )
    t_gen = time.time() - t1

    # Trim the input tokens from output
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )[0]

    return output_text.strip(), t_prep, t_gen

# Test on videos
for it in test_items:
    vp = it["local_mp4"]
    try:
        cap, tprep, tgen = generate_caption_for_path(vp)
        print(f"Video: {os.path.basename(vp)}")
        print(f"Time: {tprep:.2f}s prep + {tgen:.2f}s gen = {tprep+tgen:.2f}s total")
        print(f"Caption: {cap}\n")
    except Exception as e:
        print(f"Video: {os.path.basename(vp)}")
        print(f"❌ Error: {e}\n")
        import traceback
        traceback.print_exc()

print("✅ Sanity test complete!")

Testing on 5 videos

Video: 3e1afda20f104270_-2uIa-XMJC0.5.3.mp4
Time: 1.54s prep + 30.36s gen = 31.90s total
Caption: The camera smoothly tracks a person skateboarding on a glass platform, initially positioned in the upper right. As they move left, the camera pans left to keep them centered, maintaining a steady and fluid motion throughout.

Video: d9faf33e350a923c_0TqQje61Hoo.3.1.mp4
Time: 0.44s prep + 22.37s gen = 22.82s total
Caption: The camera arcs counterclockwise with a smooth, steady motion, exhibiting minimal shaking.

Video: 0ab6b40c0b725ae6_0BmANHSUbJg.3.5.mp4
Time: 0.54s prep + 25.80s gen = 26.34s total
Caption: The camera moves slowly backward with a slight upward motion, maintaining a very smooth and steady trajectory without any shaking.

Video: 693c83b4ba7607f7_1018.1.7.mp4
Time: 0.59s prep + 26.76s gen = 27.35s total
Caption: The camera smoothly moves backward, leading the subject from the front with minimal shaking.

Video: 7c7ee1bea4808828_2254244.3.mp4
Time: 0.59s 

In [ ]:
# Cell 7: ROBUST chunked captioning - Handles memory & long videos

from google.colab import drive
from qwen_vl_utils import process_vision_info
from tqdm import tqdm
from datasets import load_dataset
import gc
import json
import os
import torch

drive.mount('/content/drive')

ROOT = "/content/camerabench"
OUTS = f"{ROOT}/outputs"
MANIFEST_PATH = f"{OUTS}/gif2mp4_manifest.json"
DRIVE_FOLDER = "/content/drive/MyDrive/Deep Learning Fall 2025"
JSONL_PATH = f"{DRIVE_FOLDER}/motion_captions_qwen_gif.jsonl"

os.makedirs(DRIVE_FOLDER, exist_ok=True)

VIDEOS_PER_CHUNK = 100

print("Loading manifest and dataset...")
with open(MANIFEST_PATH, "r") as f:
    manifest = json.load(f)

ds = load_dataset("syCen/CameraBench", split="test")

done = set()
if os.path.exists(JSONL_PATH):
    print(f"Found existing progress file in Google Drive!")
    with open(JSONL_PATH, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                if "row_idx" in obj:
                    done.add(int(obj["row_idx"]))
            except:
                pass

processable = [m for m in manifest if m.get("status") == "ok" and int(m["row_idx"]) not in done]

total_done = len(done)
total_remaining = len(processable)
total_videos = len([m for m in manifest if m.get("status") == "ok"])

print(f"\n{'='*70}")
print(f"CAPTIONING STATUS (Google Drive)")
print(f"{'='*70}")
print(f"✅ Already completed: {total_done} videos")
print(f"📝 Remaining: {total_remaining} videos")
print(f"🎯 Total readable videos: {total_videos}")
print(f"\n▶️  This run will process: {min(VIDEOS_PER_CHUNK, total_remaining)} videos")
print(f"💾 Progress file: {JSONL_PATH}")
print(f"{'='*70}\n")

if total_remaining == 0:
    print("🎉 All videos already captioned! Run Cell 9 to export to Excel.")
else:
    chunk_to_process = processable[:VIDEOS_PER_CHUNK]

    ok_cnt, skip_cnt = 0, 0

    def caption_video(vpath: str, max_frames: int = 32) -> str:
        """Generate caption with fallback for problematic videos."""
        messages = [{
            "role": "user",
            "content": [
                {"type": "video", "video": vpath, "fps": 6.0, "max_frames": max_frames},
                {"type": "text", "text": "Describe the camera motions."},
            ],
        }]

        text = processor.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        image_inputs, video_inputs = process_vision_info(messages)

        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )

        inputs = inputs.to(model.device)

        generated_ids = model.generate(
            **inputs,
            max_new_tokens=128,
        )

        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]

        output_text = processor.batch_decode(
            generated_ids_trimmed,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0]

        return output_text.strip()

    # Process chunk
    with open(JSONL_PATH, "a") as jf:
        for m in tqdm(chunk_to_process, desc=f"Captioning (chunk of {len(chunk_to_process)})"):
            idx = int(m["row_idx"])

            # Clear memory BEFORE processing each video
            torch.cuda.empty_cache()
            gc.collect()

            try:
                # Try with normal settings first
                cap = caption_video(m["local_mp4"], max_frames=32)

            except RuntimeError as e:
                # If OOM or token length error, try with fewer frames
                if "out of memory" in str(e).lower() or "token" in str(e).lower():
                    tqdm.write(f"⚠️  Video {idx} too large, retrying with 16 frames...")
                    torch.cuda.empty_cache()
                    gc.collect()

                    try:
                        cap = caption_video(m["local_mp4"], max_frames=16)
                        tqdm.write(f"✅ Video {idx} succeeded with reduced frames")
                    except Exception as e2:
                        # Still failed, skip this video
                        tqdm.write(f"❌ Video {idx} failed even with 16 frames: {e2}")
                        skip_cnt += 1
                        continue
                else:
                    # Different error, skip
                    tqdm.write(f"❌ Video {idx} error: {e}")
                    skip_cnt += 1
                    continue

            except Exception as e:
                tqdm.write(f"❌ Video {idx} error: {e}")
                skip_cnt += 1
                continue

            # If we got here, we have a caption - save it!
            try:
                row = ds[idx]

                rec = {
                    "row_idx": idx,
                    "id_or_video_path": row.get("path", None),
                    "video_link": m.get("gif_url", None),
                    "local_mp4": m.get("local_mp4", None),
                    "caption_generated": cap,
                    "labels": row.get("labels", None),
                    "human_motion_caption": row.get("caption", None),
                    "model": "chancharikm/qwen2.5-vl-7b-cam-motion",
                    "fps": 6.0,
                    "max_frames": 32,
                    "source": "gif_preview_converted_to_mp4",
                }

                jf.write(json.dumps(rec, ensure_ascii=False) + "\n")
                jf.flush()
                ok_cnt += 1

            except Exception as e:
                tqdm.write(f"❌ Failed to save video {idx}: {e}")
                skip_cnt += 1

    print(f"\n{'='*70}")
    print(f"✅ CHUNK COMPLETE - SAVED TO GOOGLE DRIVE!")
    print(f"{'='*70}")
    print(f"Successfully captioned this run: {ok_cnt} videos")
    print(f"Errors/skipped this run: {skip_cnt} videos")
    print(f"Total completed (all runs): {total_done + ok_cnt} / {total_videos}")
    print(f"Remaining: {total_remaining - ok_cnt} videos")
    print(f"\n💾 Progress permanently saved to Google Drive")

    if total_remaining - ok_cnt > 0:
        print(f"\n▶️  To continue: Run Cell 7 again")
        print(f"   (Skipped videos will be retried automatically)")
    else:
        print(f"\n🎉 ALL VIDEOS CAPTIONED! Run Cell 9 to export to Excel.")

ValueError: mount failed

In [ ]:
# Cell 9: Export to Excel

from google.colab import drive
import pandas as pd
import json
import os

# Mount Google Drive
drive.mount('/content/drive')

JSONL_PATH = "/content/camerabench/outputs/motion_captions_qwen_gif.jsonl"
SAVE_XLSX = "/content/drive/MyDrive/Deep Learning Fall 2025/motion_captions_qwen.xlsx"

print("Reading JSONL file...")
rows = []
with open(JSONL_PATH, "r") as f:
    for line in f:
        s = line.strip()
        if not s:
            continue
        try:
            obj = json.loads(s)
            rows.append({
                "row_idx": obj.get("row_idx"),
                "video_link": obj.get("video_link"),
                "caption_generated": obj.get("caption_generated"),
                "id_or_video_path": obj.get("id_or_video_path"),
                "labels": obj.get("labels"),
                "human_motion_caption": obj.get("human_motion_caption"),
            })
        except Exception as e:
            print(f"⚠️  Skipping malformed line: {e}")

# Create DataFrame
df = pd.DataFrame(rows)

# Save to Excel
print(f"Saving to Excel...")
os.makedirs(os.path.dirname(SAVE_XLSX), exist_ok=True)

with pd.ExcelWriter(SAVE_XLSX, engine="xlsxwriter") as writer:
    df.to_excel(writer, index=False, sheet_name="captions")

    # Auto-adjust column widths
    worksheet = writer.sheets["captions"]
    for i, col in enumerate(df.columns):
        max_len = max(df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.set_column(i, i, min(max_len, 50))

print(f"\n{'='*70}")
print(f"✅ EXCEL EXPORT COMPLETE!")
print(f"{'='*70}")
print(f"Total captions: {len(df)}")
print(f"Saved to: {SAVE_XLSX}")
print(f"\n📊 Preview:")
print(df.head(3)[["row_idx", "caption_generated"]])

In [ ]:
# --- Colab: Progress polling & peek ---
!wc -l /content/camerabench/outputs/motion_captions_qwen_gif.jsonl || echo "No JSONL yet."

# Print first 3 non-empty, well-formed lines
import json
path = "/content/camerabench/outputs/motion_captions_qwen_gif.jsonl"
try:
    with open(path, "r") as f:
        shown = 0
        for line in f:
            s = line.strip()
            if not s:
                continue
            try:
                obj = json.loads(s)
                print(json.dumps(obj, indent=2, ensure_ascii=False))
                shown += 1
                if shown >= 3:
                    break
            except Exception:
                continue
except FileNotFoundError:
    print("JSONL not found.")

In [ ]:
# --- Colab: Export to Excel (Drive) ---
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

JSONL_PATH = "/content/camerabench/outputs/motion_captions_qwen_gif.jsonl"
SAVE_XLSX = "/content/drive/MyDrive/Deep Learning Fall 2025/scene_captions_qwen.xlsx"

rows = []
with open(JSONL_PATH, "r") as f:
    for line in f:
        s = line.strip()
        if not s:
            continue
        try:
            obj = json.loads(s)
            rows.append({
                "video_link": obj.get("video_link"),
                "caption_generated": obj.get("caption_generated"),
                "id_or_video_path": obj.get("id_or_video_path"),
            })
        except Exception:
            # skip malformed lines
            pass

df = pd.DataFrame(rows, columns=["video_link", "caption_generated", "id_or_video_path"])
os.makedirs(os.path.dirname(SAVE_XLSX), exist_ok=True)
with pd.ExcelWriter(SAVE_XLSX, engine="xlsxwriter") as writer:
    df.to_excel(writer, index=False, sheet_name="captions")

print("Saved Excel to:", SAVE_XLSX)
print(df.head(3))